In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.decomposition import PCA
from GA_functions import fitness, select_parents, crossover, mutation, GA_algorithm, conc_to_spectra, normalize_and_pca

## Many iterations

In [2]:
#This block of code creates random concentrations and spectra to start the algorithm. 
Sample_concentrations = pd.read_excel('Sample_concentrations.xlsx')
red_conc = Sample_concentrations[['Red vol frac']].values
green_conc = Sample_concentrations[['Green vol frac']].values
blue_conc = Sample_concentrations[['Blue vol frac']].values
sample_conc = np.hstack((red_conc, green_conc, blue_conc))

sample_spectra = pd.read_excel('Sample_spectra.xlsx')
sample_spectra = np.asarray(sample_spectra)
red = sample_spectra[:,1]
green = sample_spectra[:,2]
blue = sample_spectra[:,3]
wavelength = sample_spectra[:,0]

conc_array = np.random.dirichlet((1, 1, 1), 10)
spectra_array = conc_to_spectra(conc_array, red, green, blue, wavelength)

FileNotFoundError: [Errno 2] No such file or directory: 'Sample_concentrations.xlsx'

In [ ]:
#best_conc_lst = []
#for i in range(20):


y_train = sample_conc[0:-1,:]
x_train = sample_spectra[:,1:-1]
objective_row = -1 #Change this number for a different concentration to test. 
x_test = sample_spectra[:,objective_row].reshape(-1,1) 
print('The target conc is:', sample_conc[objective_row,:])
print(conc_array)
print()

######## Use this if the starting concentrations and spectra from excel are to be used ###########
#y_train = sample_conc[0:-1,:]
#y_train_conc = y_train
##################################################################################################


######## Use this for random starting concentrations and spectra #################################
x_train = spectra_array.T
y_train_conc = conc_array 
##################################################################################################


#Normalizae x_train
x_train = MinMaxScaler().fit(x_train).transform(x_train)
x_test = MinMaxScaler().fit(x_test).transform(x_test).T
x_train = x_train.T
x_test = x_test.reshape(1,-1)[0].reshape(-1,1).T

pca = PCA(n_components=3)
x_train_new = pca.fit(x_train)
x_train_spectra = pca.transform(x_train)
x_test = pca.transform(x_test)

best_gen0 = fitness(x_train_spectra, y_train_conc, x_test)
best_conc = best_gen0[-1,x_train_spectra.shape[1]:-1]

gen_concs = [y_train_conc]
best_conc_list = [best_conc]
for i in range(40):
    gen2_spectra = conc_to_spectra(gen_concs[i], red, green, blue, wavelength)
    #gen2_spectra = normalize_and_pca(gen2_spectra)
    x_train = gen2_spectra.T
    x_train = MinMaxScaler().fit(x_train).transform(x_train)
    x_train = x_train.T

    gen2_spectra = pca.transform(x_train)
     
    gen3 = GA_algorithm(gen2_spectra, gen_concs[i], x_test, 20, 10, 0.8)
    gen_concs.append(gen3)
    print(gen3)
    print()
    #best_gen5 = fitness(gen2_spectra, gen_concs[i], x_test)
    #best_conc = best_gen5[-1,3:-1]
    #best_conc_list.append(best_conc)


In [ ]:
mean_red = []
mean_green = []
mean_blue = []

for i in range(len(gen_concs)):
    mean_row = np.mean(gen_concs[i], axis = 0)
    mean_red.append(mean_row[0])
    mean_green.append(mean_row[1])
    mean_blue.append(mean_row[2])
print(np.mean(mean_red))
print(np.mean(mean_green))
print(np.mean(mean_blue))

    

In [ ]:
np.mean(gen3, axis=0)

In [ ]:
x_test